# 2. Kafka Producer - Simple Sample Data

Produces simple sample messages to a topic in the MSK cluster.

In [ ]:
import json
import boto3
from kafka import KafkaProducer
from kafka.errors import KafkaError

In [ ]:
# Get Kafka bootstrap servers from SSM
ssm = boto3.client('ssm')
bootstrap_servers = ssm.get_parameter(Name='/kafka/bootstrap_servers', WithDecryption=True)['Parameter']['Value']
print(f"Bootstrap servers: {bootstrap_servers}")

In [ ]:
# Create producer
producer = KafkaProducer(
    bootstrap_servers=bootstrap_servers.split(','),
    value_serializer=lambda v: json.dumps(v).encode('utf-8'),
    key_serializer=lambda k: k.encode('utf-8') if k else None
)

## Produce sample messages

In [ ]:
TOPIC = 'sample-topic'

sample_messages = [
    {'id': 1, 'message': 'Hello Kafka', 'timestamp': '2024-01-15T10:00:00'},
    {'id': 2, 'message': 'Sample data for CDC testing', 'timestamp': '2024-01-15T10:01:00'},
    {'id': 3, 'message': 'Debezium will capture changes', 'timestamp': '2024-01-15T10:02:00'},
]

for msg in sample_messages:
    future = producer.send(TOPIC, value=msg, key=str(msg['id']))
    try:
        record_metadata = future.get(timeout=10)
        print(f"✓ Sent: {msg['message']} -> partition {record_metadata.partition}, offset {record_metadata.offset}")
    except KafkaError as e:
        print(f"✗ Failed: {e}")

producer.flush()
print(f"\nDone. {len(sample_messages)} messages sent to topic '{TOPIC}'")

## Optional: Produce custom message

In [ ]:
# Send a custom message
custom = {'id': 99, 'message': 'My custom message', 'timestamp': '2024-01-15T12:00:00'}
producer.send(TOPIC, value=custom, key='99')
producer.flush()
print(f"Sent: {custom}")

In [ ]:
producer.close()